In [1]:
# Setup mlflow tracking
import mlflow
url = '127.0.0.1'
port = '8080'

mlflow.set_tracking_uri(uri=f"http://{url}:{port}")
print(f"MLflow tracking uri set to: {mlflow.get_tracking_uri()}")

MLflow tracking uri set to: http://127.0.0.1:8080


In [ ]:
# Data processing
import datetime
import utils
import json

# Load API key
polygon_api_key = json.load(open('keys.json'))['POLYGON_API_KEY']

# Get min date
min_date = utils.get_max_date(file_dir='data/daily-aggregates')

# First time min date
# min_date = (datetime.datetime.now() - datetime.timedelta(days=5 * 365)).strftime('%Y-%m-%d')

# Download stock data
utils.update_stock_data(
    min_date=min_date.strftime('%Y-%m-%d'),
    max_date=datetime.datetime.now().strftime('%Y-%m-%d'),
    api_key=polygon_api_key
)

In [ ]:
import pandas as pd

stock_df = utils.create_stock_dataframe(file_dir='data/daily-aggregates')
# Sort data by date
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.sort_values(by=['Date'], inplace=True, ascending=False)

# Save dataframe
stock_df.to_csv('data/stock_data/data.csv', index=False)

In [ ]:
null_df = stock_df.groupby(['Date']).agg({'Transactions': lambda x: x.isnull().sum()})

null_df.reset_index(inplace=True)
null_df = null_df.rename(columns = {'index': 'Date'})

import plotly.express as px

fig = px.line(null_df, x='Date', y='Transactions')
fig.show()

In [ ]:
null_df = stock_df.groupby(['Date']).agg({'Close_Price': lambda x: x.isnull().sum()})

null_df.reset_index(inplace=True)
null_df = null_df.rename(columns = {'index': 'Date'})

import plotly.express as px

fig = px.line(null_df, x='Date', y='Close_Price')
fig.show()

In [ ]:
# Available summary data
from matplotlib import pyplot as plt

vals = stock_df.Exchange_Symbol.value_counts()
plt.hist(vals)
plt.title('Number of Daily Summaries for Stocks')
plt.xlabel('Daily Summaries')
plt.ylabel('Frequency')
plt.show()

In [2]:
import mlflow
import utils
import os
import json
import datetime

import numpy as np
import pandas as pd
import multiprocessing as mp

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from mlflow.models import infer_signature

# Load stock data if not already loaded
try:
    stock_df
except Exception as e:
    print(e)
    stock_df = pd.read_csv('data/stock_data/data.csv')
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Limit stocks to those with at least 1000 daily summaries
forecast_symbols = stock_df.Exchange_Symbol.value_counts()[(stock_df.Exchange_Symbol.value_counts() > 1000).values]
forecast_symbols = list(forecast_symbols.keys())

# Place holder for parameters
param_list = []
for symbol in forecast_symbols:
    param_list.append({
        'stock_symbol': symbol,
        'df': stock_df
    })

# params = []
pool = mp.Pool(mp.cpu_count()-1)
params = pool.map(utils.create_forecast_params, param_list)
pool.close()


# with mlflow.start_run(run_name=f"stock-predictions-{datetime.datetime.now().strftime('%Y%m%d')}") as parent_run:
#     pool = mp.Pool(processes=mp.cpu_count()-1)
#     pool.map(train_model, params)
# pool.close()


name 'stock_df' is not defined
Creating forecast parameters for NVDA...
Creating forecast parameters for FTSL...
Creating forecast parameters for EC...
Creating forecast parameters for HGV...
Creating forecast parameters for VOE...
Creating forecast parameters for AEE...
Creating forecast parameters for RIGL...
Creating forecast parameters for RBBN...
Creating forecast parameters for SAR...
Creating forecast parameters for ORN...
Creating forecast parameters for VUSE...
Creating forecast parameters for IWO...
Creating forecast parameters for LMNR...
Creating forecast parameters for SCSC...
Creating forecast parameters for LOGI...
Creating forecast parameters for CGW...
Creating forecast parameters for FLSW...
Creating forecast parameters for FNV...
Creating forecast parameters for EBAY...
Creating forecast parameters for CRBP...
Creating forecast parameters for DUST...
Creating forecast parameters for NVRO...
Creating forecast parameters for ITEQ...
Creating forecast parameters for CCL

In [3]:
with mlflow.start_run(run_name=f"stock-predictions-{datetime.datetime.now().strftime('%Y%m%d')}") as parent_run:
    pool = mp.Pool(processes=mp.cpu_count()-1)
    pool.map(utils.train_model, params)
pool.close()

/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)


Fitting model for NVDA...
Fitting model for SAR...
Fitting model for CCL...


/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', in

Fitting model for EVTC...
Fitting model for ACV...
Fitting model for ALX...
Fitting model for OPI...
Fitting model for THG...
Fitting model for BMRC...
Fitting model for EMMF...
Fitting model for CLH...
Fitting model for UA...


/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
20:38:19 - cmdstanpy - INFO - Chain [1] start processing
20:38:19 - cmdstanpy - INFO - Chain [1] start processing
/Users/jacobdym/Documents/streamlit/streamlit_stock_app/utils.py:323: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method='ffill', inplace=True)
20:38:19 - cmdstanpy - INFO - Chain [1] start processing
20:38:19 - cmdstanpy - INFO - Chain [1] start processing
20:38:19 - cm

...

Fitting model for BKIE...


20:39:21 - cmdstanpy - INFO - Chain [1] done processing
20:39:21 - cmdstanpy - INFO - Chain [1] start processing
 82%|████████▏ | 18/22 [00:08<00:02,  1.81it/s]20:39:21 - cmdstanpy - INFO - Chain [1] done processing
20:39:21 - cmdstanpy - INFO - Chain [1] done processing
 83%|████████▎ | 25/30 [00:24<00:06,  1.21s/it]20:39:21 - cmdstanpy - INFO - Chain [1] done processing
20:39:21 - cmdstanpy - INFO - Chain [1] done processing
 80%|████████  | 24/30 [00:10<00:03,  1.82it/s]20:39:21 - cmdstanpy - INFO - Chain [1] done processing
20:39:21 - cmdstanpy - INFO - Chain [1] done processing
20:39:21 - cmdstanpy - INFO - Chain [1] start processing
 90%|█████████ | 27/30 [00:17<00:02,  1.16it/s]20:39:21 - cmdstanpy - INFO - Chain [1] start processing
20:39:21 - cmdstanpy - INFO - Chain [1] start processing
20:39:21 - cmdstanpy - INFO - Chain [1] done processing
 67%|██████▋   | 14/21 [00:07<00:04,  1.72it/s]20:39:21 - cmdstanpy - INFO - Chain [1] start processing
20:39:21 - cmdstanpy - INFO - Ch

Fitting model for SLQT...


 71%|███████▏  | 15/21 [00:07<00:03,  1.56it/s]20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] done processing
 83%|████████▎ | 25/30 [00:11<00:03,  1.44it/s]20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
  5%|▌         | 1/19 [00:00<00:05,  3.49it/s]20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
20:39:22 - cmdstanpy - INFO - Chain [1] done processing
20:39:22 - cmdstanpy - INFO - Chain [1] start processing
 87%|███████

MlflowException: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}

In [ ]:
with mlflow.start_run():
    prophet_model.fit(train_df)

    # extract and log parameters such as changepoint_prior_scale in the mlflow run
    model_params = {
        name: value for name, value in vars(prophet_model).items() if np.isscalar(value)
    }
    mlflow.log_params(model_params)

    # cross validate with 900 days of data initially, predictions for next 30 days
    # walk forward by 30 days
    cv_results = cross_validation(
        prophet_model, initial="900 days", period="30 days", horizon="30 days"
    )

    # Calculate metrics from cv_results, then average each metric across all backtesting windows and log to mlflow
    cv_metrics = ["mse", "rmse", "mape"]
    metrics_results = performance_metrics(cv_results, metrics=cv_metrics)
    average_metrics = metrics_results.loc[:, cv_metrics].mean(axis=0).to_dict()
    mlflow.log_metrics(average_metrics)

    # Calculate model signature
    train = prophet_model.history
    predictions = prophet_model.predict(prophet_model.make_future_dataframe(30))
    signature = infer_signature(train, predictions)

    model_info = mlflow.prophet.log_model(
        prophet_model, "prophet-model", signature=signature
    )

In [ ]:
# Load saved model
prophet_model_saved = mlflow.pyfunc.load_model(model_info.model_uri)
val_df['floor'] = train_df['y'].min()
val_df['t'] = prophet_model_saved.predict(val_df)['trend']
val_df['y_scaled'] = val_df['y'] / val_df['t']
predictions = prophet_model_saved.predict(val_df)